In [200]:
trainer_party_fp = "../src/data/trainer_parties.h"

In [201]:
import re
import random

In [213]:
with open(trainer_party_fp, "r") as f:
    tp_source = f.read()

In [214]:
start_token = "// Start of actual trainer data"
start_index = tp_source.find(start_token) + len(start_token) + 1

In [215]:
parties = tp_source[start_index:].split(";\n")
print(len(parties))
to_remove = []
for p in parties:
    if "DUMMY" in p:
        to_remove.append(p)
    if "CustomMoves" in p:
        to_remove.append(p)
    if "Rival" in p:
        to_remove.append(p)
for r in to_remove:
    if r in parties:
        parties.remove(r)
print(len(parties))

654
509


In [205]:
def extract_species(party: str) -> list:
    species = []
    start_index = party.find("SPECIES")
    while start_index != -1:
        comma_index = party.find(",", start_index)
        species.append(party[start_index:comma_index])
        start_index = party.find("SPECIES", start_index + 1)
    return species

def get_num_species(party: str) -> int:
    return party.count("SPECIES")

def get_avg_level(party: str) -> int:
    level_token = ".lvl = "
    level_sum = 0
    level_count = 0
    start_index = party.find(level_token)
    while start_index != -1:
        end_index = party.find(",", start_index)
        level_sum += int(party[start_index + len(level_token):end_index])
        start_index = party.find(level_token, start_index + 1)
        level_count += 1
    return int(level_sum / level_count)

def get_moves_type(party: str) -> str:
    default_moves_token = "DefaultMoves"
    if default_moves_token in party:
        return "default"
    return "custom"

In [212]:
def get_compatible_pokemon(party_type: str, avg_level: int, file_path: str = "trainer_source_list.txt") -> list: 
    with open(file_path, "r") as f:
        source_list = f.read()
    start_index_token = f"[{party_type.capitalize()}]\n"
    start_index = source_list.find(start_index_token)
    if start_index == -1:
        print(f"Error: {party_type} not found in source list")
        return []
    end_index = source_list.find("\n\n", start_index)
    source_list = source_list[start_index+len(start_index_token):end_index].split("\n")
    species = []
    for s in source_list:
        (name, level) = s.split("|")
        if int(level) <= avg_level:
            species.append(f"SPECIES_{name.upper()}")
    return species

def get_amount_new_pokemon(party: str) -> int:
    current_amount = get_num_species(party)
    max_additions = 6 - current_amount
    return random.randint(0, max_additions)

def get_random_level(avg_level: int) -> int:
    level = random.randint(avg_level - 5, avg_level + 5)
    if level < 1:
        return 1
    if level > 100:
        return 100
    return level

def identify_party_type(party: str) -> str:
    id_list_fp = "trainer_id_list.txt"
    with open(id_list_fp, "r") as f:
        id_list = f.read()
    
    trainer_id_pokes = {}
    trainer_id_scores = {}
    labels = re.findall(r"\[.*\]", id_list)
    for l in labels:
        trainer_id_scores[l[1:-1]] = 0
        start_index_token = f"{l}\n"
        start_index = id_list.find(start_index_token)
        end_index = id_list.find("\n\n", start_index)
        source_list = id_list[start_index+len(start_index_token):end_index].split("\n")
        species = []
        for s in source_list:
            species.append(f"SPECIES_{s.upper()}")
        trainer_id_pokes[l[1:-1]] = species
    current_pokes = extract_species(party)
    for p in current_pokes:
        for k, v in trainer_id_pokes.items():
            if p in v:
                trainer_id_scores[k] += 1
    max_score = 0
    max_key = ""
    for k, v in trainer_id_scores.items():
        if v > max_score:
            max_score = v
            max_key = k
    if max_key == "":
        return None
    return max_key

def random_pokemon(party_type: str, avg_level: int):
    species_list = get_compatible_pokemon(party_type, avg_level)
    species = random.choice(species_list)
    level = get_random_level(avg_level)
    return (species, level)

def build_pokemon(species: str, level: int) -> str:
    iv = random.randint(0, 31)
    return f"{{ \n    .iv = {iv},\n    .lvl = {level},\n    .species = {species},\n }}"

def regenerate_party(party: str) -> str:
    avg_level = get_avg_level(party)
    party_type = identify_party_type(party)
    if party_type is None:
        return party + ";"
    num_pokes = get_amount_new_pokemon(party) + get_num_species(party)
    new_party = party.split(" = ")[0] + " = {"
    for i in range(num_pokes):
        species, level = random_pokemon(party_type, avg_level)
        new_party += f"\n    {build_pokemon(species, level)},"
    new_party = new_party[:-1] + "\n};"
    return new_party

def get_party_header(party: str) -> str:
    return party.split(" = ")[0]

def replace_party(new_party: str, party_code: str) -> str:
    header = get_party_header(new_party)
    header_index = party_code.find(header)
    end_index = party_code.find("};", header_index)
    first_chunk = party_code[:header_index]
    second_chunk = party_code[end_index + 2:]
    party_code = first_chunk + new_party + second_chunk
    return party_code


In [216]:
for party in parties:
    if party == "":
        continue
    new_party = regenerate_party(party)
    tp_source = replace_party(new_party, tp_source)

In [217]:
for p in parties:
    if "Jacki" in p:
        print(p)
print(parties[0])


static const struct TrainerMonItemDefaultMoves sParty_LadyJacki[] = {
    {
        .iv = 0,
        .lvl = 48,
        .species = SPECIES_HOPPIP,
        .heldItem = ITEM_STARDUST,
    },
    {
        .iv = 0,
        .lvl = 50,
        .species = SPECIES_SKIPLOOM,
        .heldItem = ITEM_STARDUST,
    },
}
static const struct TrainerMonNoItemDefaultMoves sParty_YoungsterBen[] = {
    {
        .iv = 0,
        .lvl = 11,
        .species = SPECIES_RATTATA,
    },
    {
        .iv = 0,
        .lvl = 11,
        .species = SPECIES_EKANS,
    },
}


In [218]:
with open(trainer_party_fp, "w") as f:
    f.write(tp_source)

In [ ]:
all_species = []
default_moves_parties = 0
for party in parties:
    moves_type = get_moves_type(party)
    species = extract_species(party)
    if moves_type == "default":
        default_moves_parties += 1
    for s in species:
        if s not in all_species:
            all_species.append(s)

In [34]:
default_moves_parties

536

In [52]:
get_compatible_pokemon("fighter", 1)

['SPECIES_MANKEY',
 'SPECIES_MACHOP',
 'SPECIES_AIPOM',
 'SPECIES_TEDDIURSA',
 'SPECIES_TYROGUE',
 'SPECIES_SLAKOTH',
 'SPECIES_MAKUHITA',
 'SPECIES_MEDITITE']

In [73]:
get_amount_new_pokemon(parties[0])

3